# Introduction to Markov Chains

A __Markov chain__ is a probabilistic model of a sequence of events. The probability of each event depends only on the previous event. 

You may have encountered Markov chains in linear algebra class, because they are applications of important concepts in linear algebra. 

In this notebook, we will use Python to do Markov chain calcuations. We will need some linear algebra functions provided by the numpy package, so go ahead and import it:

In [0]:
import numpy

## Example: Weather in Oz

In the land of Oz, each day brings exactly one of the following types of weather:

* Nice
* Rain
* Snow

The weather changes every day, with the following probabilities:

* If today is Nice, then tomorrow will be either Rain or Snow with equal probability.
* If today is Rain, then the probability of Rain tomorrow is 1/2, the probability of Nice tomorrow is 1/4, and the probability of snow tomorrow is 1/4.
* If today is Snow, then the probability of Snow tomorrow is 1/2, the probability of Nice tomorrow is 1/4, and the probability of Rain tomorrow is 1/4.

To simulate the weather in Oz, we can define the __Transition Matrix__ $P$. This is a $3 \times 3$ matrix, since there are three types of weather. The columns and rows correspond to _Nice_, _Rain_, and _Snow_, in that order. The entry in row $i$, column $j$ gives the probability that, if the weather today is $j$, then the weather tomorrow will be $i$. Here is the matrix:

$$ P = \left[ \begin{array}{ccc} 0.0 & 0.25 & 0.25 \\ 0.5 & 0.5 & 0.25 \\ 0.5 & 0.25 & 0.5 \end{array}\right]$$

Note that each column sums to 1 (but each row does not).

__Question:__ _In the long run, what is the probability of each type of weather on a given day?_

Let's examine this question from three different perspectives. Before we do this, store the transition matrix in Python's memory:

In [0]:
transProb = numpy.array([[0,0.25,0.25],[0.5,0.5,0.25],[0.5,0.25,0.5]])
transProb

NameError: ignored

## Perspective 1: Simulation

Let's simulate the weather for the next 10,000 days, according to the transition probabilities.

Suppose that today's weather is Nice. Then the probability of each type of weather tomorrow is given by the first column of the transition matrix. The following code shows one way of randomly generating tomorrow's weather according to these probabilities. The second argument, 1, means that we want to choose one item from the array given in the first argument. 

In [0]:
numpy.random.choice(["Nice","Rain","Snow"], 1, p=transProb[:,0])

array(['Snow'], dtype='<U4')

Working with the text strings "Nice", "Rain", and "Snow" is not convenient. Let's replace each weather type by the index of its column in the transition matrix, remembering that Python starts indexes at zero.

If today's weather is 0 (Nice), then our simulation for tomorrow's weather becomes:

In [0]:
numpy.random.choice(3, 1, p=transProb[:,0])

array([2])

Since tomorrow's weather is given by a single number, we don't really need it in an array. We will get the number out of the array:

In [0]:
tomorrow = numpy.random.choice(3, 1, p=transProb[:,0])[0]
tomorrow

2

The weather the day after tomorrow is determined as follows

In [0]:
probs = transProb[:,tomorrow]
numpy.random.choice(3, 1, p=probs)[0]

2

Since we want to do this over and over, let's write a loop. The following code simulates the weather for lots of days, storing each day's weather in an array called `allWeather`. 

When the simulation finishes, we use `numpy.bincount` to find the number of days of each weather type. We then divide by the total number of days to get the percentage of days that each weather type occurs.

In [0]:
numDays = 100000
currWeather = 0  # start with Nice
allWeather = numpy.zeros(numDays, dtype=numpy.int64)  # we will store each day's weather here

for i in range(numDays):
  allWeather[i] = currWeather
  probs = transProb[:,currWeather]
  currWeather = numpy.random.choice(3, 1, p=probs)[0]

numpy.bincount(allWeather)/numDays


array([0.19975, 0.3989 , 0.40135])

Looks like the weather is about 20% Nice, 40% Rain, and 40% Snow.

## Perspective 2: Matrix Powers

Suppose again that today is Nice. As we said previously, the probabilities for each type of weather tomorrow are given by the first column of the transition matrix:

In [0]:
day1probs = transProb[:,0]
day1probs

array([0. , 0.5, 0.5])

We can obtain the probabilities for each type of weather on the day after tomorrow by matrix multiplication. Since our matrices and vectors are stored as numpy arrays, we can multiply them using the function numpy.matmul, or the operator @.

In [0]:
day2probs = transProb @ day1probs
day2probs

array([0.25 , 0.375, 0.375])

Do you agree that these are the probabilities of each type of weather two days from now, if today is Nice?

Similarly, we can find the probabilities of each type of weather three days from now:

In [0]:
day3probs = transProb @ day2probs
day3probs

array([0.1875 , 0.40625, 0.40625])

Or 100 days from now:

In [0]:
dayprobs = transProb[:,0]
for i in range(99):
  dayprobs = transProb @ dayprobs
dayprobs

array([0.2, 0.4, 0.4])

Interesting...these are almost the same numbers that we found in our simulation.

However, our loop wasn't very efficient, since we really just multiplied our transition matrix by itself 100 times. It would be faster to use matrix powers. That is, the probability of each type of weather on day $n$ is given by

$$P^n v$$

where $v$ is a vector that gives today's weather. If today is Nice, then $v = [1,0,0]^\top$.

Fortunately, numpy makes it easy to compute matrix powers. Here is our transition probability matrix multiped by itself:

In [0]:
numpy.linalg.matrix_power(transProb,2)

array([[0.25  , 0.1875, 0.1875],
       [0.375 , 0.4375, 0.375 ],
       [0.375 , 0.375 , 0.4375]])

Here is our transition probability matrix multiplied raised to the power 100:

In [0]:
numpy.linalg.matrix_power(transProb,100)

array([[0.2, 0.2, 0.2],
       [0.4, 0.4, 0.4],
       [0.4, 0.4, 0.4]])

We see that each column now contains the same numbers that we found before! These numbers give the __steady-state distribution__ of the weather. It doesn't matter what weather we start with; in the long run, the weather will be 20% nice, 40% rain, and 40% snow.

## Perspective 3: Eigenvectors

If $v$ is a steady-state vector of matrix $P$, then $Pv = v$. That means that $v$ is an __eigenvector__ of $P$ corresponding to __eigenvalue__ $1$.

Numpy can find eigenvalues and eigenvectors:

In [0]:
eigenStuff = numpy.linalg.eig(transProb)
eigenStuff

(array([-0.25,  1.  ,  0.25]),
 array([[-8.16496581e-01,  3.33333333e-01, -7.85046229e-17],
        [ 4.08248290e-01,  6.66666667e-01, -7.07106781e-01],
        [ 4.08248290e-01,  6.66666667e-01,  7.07106781e-01]]))

The `numpy.linalg.eig` function returns a _tuple_ containing an array of eigenvalues and an array of eigenvectors.

We can get the eigenvalues like this:

In [0]:
eigenStuff[0]

array([-0.25,  1.  ,  0.25])

We can get the eigenvectors as follows. The eigenvectors are the _columns_ of the 2-D array.

In [0]:
eigenStuff[1]

array([[-8.16496581e-01,  3.33333333e-01, -7.85046229e-17],
       [ 4.08248290e-01,  6.66666667e-01, -7.07106781e-01],
       [ 4.08248290e-01,  6.66666667e-01,  7.07106781e-01]])

Note that $1$ is an eigenvalue, as we said it should be. It happens to also be at index 1 in the array of eigenvalues, but this won't always be the case. 

The eigenvector corresponding to eigenvalue 1 is:

In [0]:
eigVec1 = eigenStuff[1][:,1]
eigVec1

array([0.33333333, 0.66666667, 0.66666667])

Numpy normalizes all eigenvectors so that their length is 1. To get the steady-state probability distribution, we want to rescale the vector so that its entries add up to 1. Here is a simple way to do it:

In [0]:
eigVec1/numpy.sum(eigVec1)

array([0.2, 0.4, 0.4])

We have found our probabilities yet again!

## Exercise

Construct a transition matrix for a game that involves moving through 8 positions (or "states"), arranged in a circle. Let $S_1$ be the first position, $S_2$ the second position, and so on, with $S_8$ the last position.

In the game, suppose that you roll a dice that tells you how many positions to move. After you reach $S_8$, the next position is $S_1$, and you go around again.

Consider two versions of the game:

* __Version 1:__ You move from one state to the next rolling a fair 3-sided die. Hence if you are in $S_3$ and roll a 2, you move to $S_5$. If you are in $S_7$ and roll a 3, you move to $S_2$.
* __Version 2:__ You roll a fair 3-sided die as in Version 1, but now if you land in $S_5$, you proceed directly to $S_1$.

Construct transition matrices and determine the steady-state distribution for each game.  In other words, what is the probability of being in a particular state, say $S_3$, after a large number of rolls?